In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import tensorflow as tf
from PIL import Image
import os
import matplotlib.pyplot as plt

In [ ]:
path = '../input/cassava-leaf-disease-classification/'

In [ ]:
train_path = pd.read_csv(path + 'train.csv')
train_path['label'] = train_path['label'].astype('string')
train_path.head(5)

In [ ]:
disease_names = pd.read_json(path + 'label_num_to_disease_map.json', typ='series')
disease_names

In [ ]:
plt.figure(figsize=(16, 12))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    image = Image.open(path + 'train_images/' + train_path.iloc[i]['image_id'])
    array = np.array(image)
    plt.imshow(array)
    label=train_path.iloc[i]['label']
    plt.title(f'{disease_names[int(label)]}')
plt.show()

In [ ]:
sizes = []
for i in range(1, len(train_path), 200):
    image = Image.open(path + 'train_images/' + train_path.iloc[i]['image_id'])
    array = np.array(image)
    sizes.append(array.shape)
print('Image size is:', set(sizes))


In [ ]:
image_width, image_height = 250, 250

In [ ]:
datagen = ImageDataGenerator(validation_split=0.2,
                             vertical_flip=True,
                             horizontal_flip=True)
train_datagen_flow = datagen.flow_from_dataframe(
    dataframe=train_path,
    directory=path + 'train_images',
    x_col='image_id',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=20)

In [ ]:
valid_datagen_flow = datagen.flow_from_dataframe(
    dataframe=train_path,
    directory=path + 'train_images',
    x_col='image_id',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=20,
    subset='validation',
    seed=12345)

In [ ]:
current_balance = train_path['label'].value_counts(normalize=True)
current_balance

In [ ]:
class_weight = {0: (1 - current_balance['0']) / (1 - current_balance.min()),
                1: (1 - current_balance['1']) / (1 - current_balance.min()),
                2: (1 - current_balance['2']) / (1 - current_balance.min()),
                3: (1 - current_balance['3']) / (1 - current_balance.min()),
                4: (1 - current_balance['4']) / (1 - current_balance.min())}

class_weight

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [ ]:
 model = Sequential()
optimizer = Adam(lr=0.001)
backbone = EfficientNetB2(include_top=False, 
                          weights=None, 
                          pooling='avg')
model.add(backbone)
model.add(Dense(5, activation='softmax'))
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizer, 
              metrics=["accuracy"])
model.fit_generator(train_datagen_flow,
                    validation_data=valid_datagen_flow, 
                    epochs=10,
                    class_weight=class_weight,
                    callbacks=[early_stopping, mc],
                    verbose=2)

In [ ]:
saved_model = load_model('best_model.h5')

In [ ]:
submission = pd.DataFrame(columns=['image_id','label'])
for image_name in os.listdir(path + 'test_images'):
    image_path = os.path.join(path + 'test_images', image_name)
    image = tf.keras.preprocessing.image.load_img(image_path)
    resized_image = image.resize((image_width, image_height))
    numpied_image = np.expand_dims(resized_image, 0)
    tensored_image = tf.cast(numpied_image, tf.float32)
    submission = submission.append(pd.DataFrame({'image_id': image_name,
                                                 'label': saved_model.predict_classes(tensored_image)}))

submission

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)